# Transfer learning

In [ ]:
import torch
from torchvision import transforms, datasets, models

from torch import nn
from torch import optim

In [ ]:
data_dir = 'Cat_Dog_data'

train_transforms = transforms.Compose([
                    transforms.RandomRotation(30),        
                    transforms.RandomResizedCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
                    transforms.Resize(255),                
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
# GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# resnet
model = models.resnet50(pretrained=True)

print(model)

In [ ]:
# freeze parameter
for param in model.parameters():
    param.requries_grad = False

# model
model.fc = nn.Sequential(nn.Linear(2048, 1024),
                         nn.ReLU(),
                         nn.Dropout(0.2),
                         nn.Linear(1024, 256),
                         nn.ReLU(),
                         nn.Dropout(0.2),
                         nn.Linear(256, 2),
                         nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.03)

model.to(device)

In [ ]:
epochs = 1

for e in range(epochs):
    running_loss = 0
    
    # train
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(images)
        loss = criterion(logps, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # test       
        test_loss = 0
        accuracy = 0
        
        with torch.no_grad():
            model.eval()
            
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
            
                logps = model.forward(images)
                test_loss += criterion(logps, labels)
                
                ps = torch.exp(logps)
                top_p, top_class = ps.topk(1, dim=1)
                
                equals = top_class == lables.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
            
            model.train()

            print("epoch: {}/{}".format(e+1, epochs),
                  "running loss: {}".format(running_loss),
                  "test loss: {}".format(test_loss/len(testloader)),
                  "accuracy: {}".format(accuracy/len(testloader))) 